# 一、tf.keras.Model 是什么？
tf.keras.Model（特别是通过子类化 Subclassing 方式）允许你高度灵活地自定义模型结构，并精确控制数据在模型内部的流向，适用于复杂、非标准结构的模型。

# 二、tf.keras.Model 使用的基本步骤
| 步骤        | 说明                                    | 代码示例                                                                                                                                                                                                                                     |
| --------- | ------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1. 定义模型类  | 继承 `tf.keras.Model`，在构造函数里定义各层        | `class MyModel(tf.keras.Model):  def __init__(self):    super().__init__()    self.dense1 = tf.keras.layers.Dense(64, activation='relu')     self.dense2 = tf.keras.layers.Dense(10, activation='softmax') ` |
| 2. 定义前向传播 | 重写 `call(self, inputs)` 方法，定义数据如何流过网络 | `    def call(self, inputs):     x = self.dense1(inputs)     return self.dense2(x) `                                                                                                                                    |
| 3. 实例化模型  | 创建模型对象                                | `  model = MyModel() `                                                                                                                                                                                                        |
| 4. 编译模型   | 指定优化器、损失函数和指标                         | `  model.compile(optimizer='adam',               loss='sparse_categorical_crossentropy',               metrics=['accuracy']) `                                                                                          |
| 5. 训练模型   | 使用 `fit` 输入训练数据                       | `  model.fit(train_x, train_y, epochs=5, batch_size=32) `                                                                                                                                                                     |
| 6. 评估与预测  | 用 `evaluate` 和 `predict`              | `  model.evaluate(test_x, test_y) preds = model.predict(new_data) `                                                                                                                                                        |


## 🔍 分点解释：
### ✅ 1. “按照自己需求搭建模型” 
使用 tf.keras.Model 子类化，你可以：\
1、自由定义 __init__ 中的层（如 Dense、Conv、LSTM、自定义层等）\
2、灵活组合各种层，构建非常规结构的模型，比如：\
多输入 / 多输出模型\
残差连接（ResNet）\
注意力机制\
自编码器（Autoencoder）\
GAN 的生成器或判别器\
强化学习中的策略网络等\
### ✅ 举例：

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()   # 自动继承父类的构造方法
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')   
        self.dense2 = tf.keras.layers.Dense(32)
        self.skip_connect = tf.keras.layers.Dense(32)  # 可以做残差连接
        self.output_layer = tf.keras.layers.Dense(10, activation='softmax')  # 输出层 多分类输出 

这比 Sequential 模型灵活得多。

### 2. “自己控制数据的流向” ✅ 正确
在 call(self, inputs) 方法中，你手动编写前向传播逻辑，决定数据如何流动。

In [ ]:
def call(self, x):
    h1 = self.dense1(x)
    h2 = self.dense2(h1)
    skip = self.skip_connect(x)  # 直接从输入引出一支路
    h2 = h2 + skip  # 实现残差连接
    return self.output_layer(h2)


你可以：\
写 if 条件判断（比如训练/推理不同行为）\
写 for 循环（如 RNN 展开）\
实现复杂的连接方式（如跳跃连接、多路径、注意力权重计算等）\
✅ 这是 Sequential 模型做不到的。

### ⚠️ 但要注意：不是唯一的“自定义”方式
Keras 提供了三种构建模型的方式，tf.keras.Model 子类化只是其中之一：\
| 方法               | 灵活性 | 易用性 | 适用场景                        |
|--------------------|--------|--------|--------------------------------|
| Sequential         | 低     | 高     | 简单的线性堆叠（层一个接一个） |
| 函数式 API（Functional API） | 中     | 中     | 有分支、共享层、多输入输出的静态图模型 |
| Model 子类化       | 高     | 低     | 动态逻辑、复杂控制流、研究型模型 |

✅ 所以你选择 Model 子类化，就是为了最大自由度。

### ✅ 举个例子说明“控制数据流向”的强大

In [ ]:
def call(self, x, training=None):
    if training:
        noise = tf.random.normal(tf.shape(x)) * 0.1
        x = x + noise  # 只在训练时加噪声（如去噪自编码器）
    return self.decoder(self.encoder(x))

这种训练/推理行为不同的逻辑，只能通过子类化实现。

### ❌ 常见误解澄清
| 误解                       | 澄清                                                         |
|----------------------------|--------------------------------------------------------------|
| “用了 Model 就不用 compile 和 fit 了”      | ❌ 错！仍然可以用 model.compile() 和 model.fit()，除非你写自定义训练循环。 |
| “所有模型都该用 Model 子类化”             | ❌ 错！简单模型用 Sequential 或 函数式 API 更清晰、更安全。               |
| “子类化模型一定能加速”                   | ❌ 错！它更灵活，但可能更难优化，有时函数式 API 编译更快。                 |

## ✅ 总结

使用 tf.keras.Model 子类化，就是为了让开发者像写普通   函数一样，自由定义模型结构和数据流动逻辑，特别适合研究、实验、复杂网络结构。

📌 建议：在能用 函数式 API 解决时优先用它（更简洁、可读性好）；只有在需要动态控制流时，才使用 Model 子类化。

# 三、完整示例代码（手把手）

In [ ]:
import tensorflow as tf

# 1. 定义模型类
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):   # 控制数据分向
        x = self.dense1(inputs)
        return self.dense2(x)

# 2. 实例化模型
model = MyModel()

# 3. 编译模型
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 假数据示范
import numpy as np
train_x = np.random.random((1000, 20))
train_y = np.random.randint(0, 10, (1000,))

test_x = np.random.random((200, 20))
test_y = np.random.randint(0, 10, (200,))

# 4. 训练模型
model.fit(train_x, train_y, epochs=5, batch_size=32)

# 5. 评估模型
model.evaluate(test_x, test_y)

# 6. 预测
sample_data = np.random.random((5, 20))
predictions = model.predict(sample_data)
print(predictions)


# 四、重点讲解
| 关键点                 | 说明                             |
| ------------------- | ------------------------------ |
| **定义层（`__init__`）** | 这里定义网络所有层，类似搭积木。所有层都以成员变量形式存在。 |
| **定义前向传播（`call`）**  | 数据流向和计算过程写在这里，必须返回模型输出。        |
| **实例化模型**           | 直接调用类，获得模型实例。                  |
| **编译模型**            | 绑定训练用的优化器、损失函数、评估指标。           |
| **训练/评估/预测**        | 使用 Keras 标准接口调用即可。             |

# 五、call 方法额外参数说明
call(self, inputs, training=False) 中：

inputs：输入张量。\
training：布尔值，表示当前是训练还是推理阶段（有些层如 Dropout, BatchNorm 需要区分）。\
你可以根据 training 参数设计训练和推理不同逻辑：

In [ ]:
def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
        x = tf.nn.dropout(x, rate=0.5)
    return self.dense2(x)

# 六、小结
| 你会获得          | 说明                                                     |
| ------------- | ------------------------------------------------------ |
| 完全自由的模型结构设计能力 | 支持任意复杂计算图，比如多输入多输出、跳跃连接、条件计算                           |
| 完整训练流程兼容      | 依然可以用 `compile`、`fit`、`evaluate`、`predict` 这些 Keras 接口 |
| 未来可扩展自定义训练步骤  | 可覆盖 `train_step` 实现复杂训练逻辑                              |


--- 

# 1. model.compile() 的作用

model.compile() 是 模型训练前的配置步骤，它主要做三件事：

- 1）指定优化器 (`optimize`)：

   - 决定参数如何更新（梯度下降、学习率等）。

- 2）指定损失函数 (`loss`)：

   - 衡量预测结果和真实标签之间的差异。

- 3）指定评估指标 (`metrics`)：

  - 在训练和测试过程中，用于监控模型表现（比如准确率）。

### 👉 简单来说：
model.compile(optimizer,loss,metrics,loss_weights) = 设定模型训练的`“规则”`。

---
## 2. 常用参数
### (1) optimizer —— 优化器

- 控制参数更新的方式。

- 可以是字符串或优化器对象：\
`SGD`         # 随机梯度下降\
`Adam`        # Adam优化器,现在主流默认的\
`RMSprop`     # RMSprop优化器

---
### (2) loss —— 损失函数
- 训练时最核心的目标函数。
- 选择取决于任务类型：
| 任务类型 | 常用损失函数                                                                            |
| ---- | --------------------------------------------------------------------------------- |
| 二分类  | `binary_crossentropy`                                                             |
| 多分类  | `categorical_crossentropy`（one-hot 标签）<br>`sparse_categorical_crossentropy`（整数标签） |
| 回归   | `mse`（均方误差）、`mae`（平均绝对误差）、`huber_loss`                                            |
| 其他   | 自定义函数 (callable)                                                                  |

---
### (3) metrics —— 评估指标

- 不影响训练，只是监控效果。

常见指标：  \
`accuracy`             ====>  # 分类准确率 \
`AUC`                   ====>   # ROC 曲线下的面积\
`Precision`, `Recall`   ====>   # 精确率、召回率\
`mae`, `mse`, `RootMeanSquaredError`  ====> # 回归指标

---
### (4) loss_weights （可选）

- 用于多输出模型，给不同输出的 loss 加权。

model.compile( optimizer='adam',loss={'out1':'mse', 'out2':'binary_crossentropy'},loss_weights={'out1':0.7, 'out2':0.3})

---
### (5) sample_weight_mode（少用）

- 控制样本加权的方式。常用于时间序列。

---
## 3.自定义损失 + 多指标 代码示例

In [ ]:
import tensorflow as tf

def my_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

model.compile(
    optimizer='adam',
    loss=my_loss,
    metrics=['mae', tf.keras.metrics.RootMeanSquaredError()]
)


# 4. 总结记忆

- optimizer → 怎么学（梯度下降方法）

- loss → 学什么（目标函数）

- metrics → 学得怎么样（评估标准）

---

# TensorFlow Keras 函数式 API 的使用方法
它是继承 tf.keras.Model 之外另一种非常常用且灵活的建模方式。

## 1、什么是函数式 API？
以函数的方式定义模型，将输入张量和输出张量“连接”起来。

适合构建非线性结构、多输入多输出的模型。

与 Sequential 不同，不用按层顺序添加，而是通过张量“流动”定义结构。

## 2、函数式 API 基本步骤和示例

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# 1. 定义输入张量（必须）
inputs = tf.keras.Input(shape=(20,))  # 输入特征维度是20

# 2. 定义网络层并连接（像搭积木，层和张量互相调用）
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

# 3. 创建模型，指定输入和输出
# name 这个参数，是给你创建的模型指定一个名字（name），方便你在后续使用时辨识和管理。
model = Model(inputs=inputs, outputs=outputs, name='functional_model')

# 4. 编译模型
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5. 训练
# 假数据示例
import numpy as np
train_x = np.random.random((1000, 20))
train_y = np.random.randint(0, 10, (1000,))

model.fit(train_x, train_y, epochs=5, batch_size=32)


# name 参数的讲解
model = Model(inputs=inputs, outputs=outputs, name='functional_model')
print(model.name) 
# 输出：functional_model

如果不指定，系统默认会给你一个名字，如 model_1。

| 作用         | 说明                                                                                                                        |
| ---------- | ------------------------------------------------------------------------------------------------------------------------- |
| **模型标识**   | 在 TensorFlow 和 Keras 中，每个模型都有一个名字，默认会自动生成类似 `model_1`、`model_2` 这样的名字。你也可以手动指定一个更有意义的名字，比如 `'functional_model'`，方便区分不同模型。 |
| **日志记录**   | 训练时 TensorBoard、日志或保存文件中，会用到这个名字来标识该模型。                                                                                   |
| **保存与加载**  | 保存模型时，文件夹或文件名通常包含模型名，便于管理多个模型。                                                                                            |
| **调试与可视化** | 画模型结构图、打印摘要时，会显示模型名字，帮助理解和调试。                                                                                             |


## 3、重要点说明
| 步骤                       | 说明                            |
| ------------------------ | ----------------------------- |
| `Input(shape=...)`       | 定义模型输入占位张量，是函数式 API 必需的第一步。   |
| 层的调用                     | 层本身是可调用的函数，输入张量进去，输出新的张量。     |
| `Model(inputs, outputs)` | 创建模型实例，必须明确输入和输出。             |
| 灵活连接                     | 层与层之间可以任意连接，支持多输入多输出、残差等复杂结构。 |
## 多输入多输出示例 


In [ ]:
# 多输入
input_a = tf.keras.Input(shape=(32,))
input_b = tf.keras.Input(shape=(64,))

x1 = layers.Dense(16, activation='relu')(input_a)
x2 = layers.Dense(16, activation='relu')(input_b)

# 把多个张量（Tensor）在指定的维度上拼接（Concatenate）成一个张量。
# 可以把它想象成 NumPy 里的 np.concatenate()，只是它是 Keras 层版本，可以直接在模型里用，并且是可训练计算图的一部分。
concat = layers.concatenate([x1, x2])
output = layers.Dense(1, activation='sigmoid')(concat)

model = Model(inputs=[input_a, input_b], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy')


## layers.concatenate() :详细解释
| 项目        | 说明                                                    |
| --------- | ----------------------------------------------------- |
| **功能**    | 将多个输入张量合并为一个张量（沿指定轴拼接）                                |
| **输入**    | 一个张量列表 `[x1, x2, ...]`，这些张量的形状必须**除了拼接的那个轴以外，其他维度一致** |
| **默认拼接轴** | `axis=-1`（最后一个维度，通常是**特征维度**）                         |
| **返回值**   | 一个新的张量，包含所有输入的特征拼接结果                                  |


## 4、函数式 API 和子类化对比
| 特点   | 函数式 API                | 子类化 Model        |
| ---- | ---------------------- | ---------------- |
| 定义方式 | 通过张量操作显式定义计算图          | 继承并重写 `call` 实现  |
| 灵活性  | 支持复杂拓扑，多输入多输出          | 最灵活，支持条件逻辑和自定义训练 |
| 可读性  | 清晰直观，结构一目了然            | 代码更灵活但可能复杂       |
| 训练接口 | 完全支持 `compile` 和 `fit` | 也支持，且可自定义训练流程    |
